Sentiment analysis on tweets for various companies, want to see how they're changing with time, and whether or not this is a good idea to choose where to throw $$

Still need to confirm tweets are organized by date, that will be done tmrw


The majority of this code is shamelessly stolen from:
https://github.com/ciurana2016/predict_stock_py/blob/master/predict_stock.py

AND 

https://github.com/Avhirup/Stock-Market-Prediction-Challenge/blob/master/Predicting%20Stock%20Prices%20Challenge.ipynb

Thanks!


In [1]:
import os
import sys
import tweepy
import requests
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from textblob import TextBlob

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
consumer_key = '5snDy5NGYkLOMjbvyhbyPR4Qa'
consumer_secret = 'vwMHegxeYn57plCuROuRQI9KXuI9m1l6NBjrUIcW8wb9T35r1J'

access_token = '98916612-tiFFa7Be1MQQUXEqMLNJBSk7mcQpM3rfSxUbJBzph'
access_token_secret = 'AG7ziWD1JGkMDH4QyPAZGYxcRjraRvHbZspzT8sKKASyh'

In [4]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
user = tweepy.API(auth)
FILE_NAME = 'SNAPFULL.csv'


In [ ]:
FILE_NAME = 'historical.csv'
def get_historical(quote):
    # Download file from google finance
    url = 'https://query1.finance.yahoo.com/v7/finance/download/'+quote+'?period1=1517621193&period2=1520040393&interval=1d&events=history&crumb=WogDzR8WkDK'
#    url = 'http://www.google.com/finance/historical?q=NASDAQ%3A'+quote+'&output=csv'
    r = requests.get(url, stream=True)

    if r.status_code != 400:
        with open(FILE_NAME, 'wb') as f:
            for chunk in r:
                f.write(chunk)

        return True


In [ ]:
#This I would prefer to grab from the web, continuously update
#in a more convenient manner. 

In [5]:
def stock_sentiment(quote, num_tweets):
    # Checks if the sentiment for our quote is
    # positive or negative, returns True if
    # majority of valid tweets have positive sentiment
    list_of_tweets = user.search(quote, count=num_tweets)
    positive, null = 0, 0

    for tweet in list_of_tweets:
       # print(tweet.text)
        #print("")
        blob = TextBlob(tweet.text).sentiment
        if blob.subjectivity == 0:
            null += 1
            next
        if blob.polarity > 0:
            positive += 1

    if positive > ((num_tweets - null)/2):
        #if over half the tweets are considered positive polarity, this is good! 
        return True

In [17]:
def stock_prediction():

    # Collect data points from csv
    dataset = []

    with open(FILE_NAME) as f:
        for n, line in enumerate(f):
            if n != 0:
                dataset.append(float(line.split(',')[1]))

    dataset = np.array(dataset)

    # Create dataset matrix (X=t and Y=t+1)
    def create_dataset(dataset):
        dataX = [dataset[n+1] for n in range(len(dataset)-2)]
        return np.array(dataX), dataset[2:]
        
    trainX, trainY = create_dataset(dataset)
    print("I'm training on input " +str(trainX) )
    print("and output " + str(trainY))
    # Create and fit Multilinear Perceptron model
    
    

    
    model = Sequential()
    model.add(Dense(8, input_dim=1, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, nb_epoch=200, batch_size=2, verbose=1)

    # Our prediction for tomorrow
    prediction = model.predict(np.array([dataset[0]]))
    result = 'The price will move from %s to %s' % (dataset[0], prediction[0][0])

    return result

In [7]:
stock_quote = input('Enter a stock quote from NASDAQ (e.j: AAPL, FB, GOOGL): ').upper()

# Check if the stock sentiment is positve
if not stock_sentiment(stock_quote, num_tweets=20):
    print('This stock has bad sentiment, please re-run the script')
    sys.exit()

    
    
# Check if we got te historical data
if not get_historical(stock_quote):
    print('Google returned a 404, please re-run the script and')
    print('enter a valid stock quote from NASDAQ')
    sys.exit()

# We have our file so we create the neural net and get the prediction
print(stock_prediction())

# We are done so we delete the csv file
os.remove(FILE_NAME)

Enter a stock quote from NASDAQ (e.j: AAPL, FB, GOOGL): SNAP


NameError: name 'get_historical' is not defined

In [ ]:
stock_sentiment('fit',20)

In [18]:
stock_prediction()

I'm training on input [13.5      17.15     20.74     19.07     18.84     18.389999 18.35
 19.66     19.84     19.309999 18.9      18.48     17.6      17.450001
 17.059999 16.77     17.5     ]
and output [17.15     20.74     19.07     18.84     18.389999 18.35     19.66
 19.84     19.309999 18.9      18.48     17.6      17.450001 17.059999
 16.77     17.5      16.83    ]


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/200
17/17 [==============================] - 0s 17ms/step - loss: 454.9725
Epoch 2/200
17/17 [==============================] - 0s 1ms/step - loss: 448.3866
Epoch 3/200
17/17 [==============================] - 0s 839us/step - loss: 442.0497
Epoch 4/200
17/17 [==============================] - 0s 920us/step - loss: 435.9949
Epoch 5/200
17/17 [==============================] - 0s 752us/step - loss: 430.0624
Epoch 6/200
17/17 [==============================] - 0s 886us/step - loss: 424.4122
Epoch 7/200
17/17 [==============================] - 0s 612us/step - loss: 418.9416
Epoch 8/200
17/17 [==============================] - 0s 655us/step - loss: 413.8578
Epoch 9/200
17/17 [==============================] - 0s 1ms/step - loss: 408.7969
Epoch 10/200
17/17 [==============================] - 0s 605us/step - loss: 404.0343
Epoch 11/200
17/17 [==============================] - 0s 934us/step - loss: 399.4296
Epoch 12/200
17/17 [==============================] - 0s 685us/step - loss: 395

17/17 [==============================] - 0s 641us/step - loss: 54.5915
Epoch 97/200
17/17 [==============================] - 0s 607us/step - loss: 50.7981
Epoch 98/200
17/17 [==============================] - 0s 789us/step - loss: 47.3697
Epoch 99/200
17/17 [==============================] - 0s 584us/step - loss: 43.9453
Epoch 100/200
17/17 [==============================] - 0s 557us/step - loss: 40.8796
Epoch 101/200
17/17 [==============================] - 0s 1ms/step - loss: 37.7619
Epoch 102/200
17/17 [==============================] - 0s 932us/step - loss: 34.9925
Epoch 103/200
17/17 [==============================] - 0s 785us/step - loss: 32.3378
Epoch 104/200
17/17 [==============================] - 0s 828us/step - loss: 29.8103
Epoch 105/200
17/17 [==============================] - 0s 658us/step - loss: 27.4837
Epoch 106/200
17/17 [==============================] - 0s 940us/step - loss: 25.2732
Epoch 107/200
17/17 [==============================] - 0s 817us/step - loss: 23.2375

17/17 [==============================] - 0s 664us/step - loss: 1.7536
Epoch 194/200
17/17 [==============================] - 0s 912us/step - loss: 1.7479
Epoch 195/200
17/17 [==============================] - 0s 610us/step - loss: 1.7576
Epoch 196/200
17/17 [==============================] - 0s 749us/step - loss: 1.7486
Epoch 197/200
17/17 [==============================] - 0s 935us/step - loss: 1.7492
Epoch 198/200
17/17 [==============================] - 0s 851us/step - loss: 1.7495
Epoch 199/200
17/17 [==============================] - 0s 663us/step - loss: 1.7523
Epoch 200/200
17/17 [==============================] - 0s 701us/step - loss: 1.7495


'The price will move from 13.35 to 13.781234'